In [44]:
# Import Splinter, BeautifulSoup, Pandas and other dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from sqlalchemy import true
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt
import sys
import pandas as pd
import json
import time
import random

In [45]:
# Setup variable for news data url
categories_url = 'https://raw.githubusercontent.com/tcurlster/Fake-News-Recognition-and-Classification-Project/main/Resources/News_Category_Dataset_v2.json'

In [46]:
# Import news data with the links to the articles
news_categories = pd.read_json(categories_url, lines = True)
news_categories.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [47]:
# Sample the dataframe for testing scraping
sample_dataframe = news_categories.sample(30)

In [48]:
sample_dataframe

,category,headline,authors,link,short_description,date
95507,ENTERTAINMENT,One Thing That's Holding You Back From Getting...,Rochelle Nelson,https://www.huffingtonpost.com/entry/how-to-fa...,,2015-03-23
272,TECH,Instagram Is Adding A 'Mute' Button For The Sa...,Sara Boboltz,https://www.huffingtonpost.com/entry/instagram...,And no one will know when you use it.,2018-05-22
38678,POLITICS,The Biggest Political Winners And Losers Of 2016,"Travis Irvine, ContributorComedian, journalist...",https://www.huffingtonpost.com/entry/the-bigge...,What victories should be hailed and what mista...,2016-12-29
111783,POLITICS,A GOP Candidate Shared Thoughts About Medical ...,Kim Bellware,https://www.huffingtonpost.com/entry/bruce-rau...,,2014-09-17
26180,BLACK VOICES,Shondaland's New Show 'For The People' Looks A...,Zahara Hill,https://www.huffingtonpost.com/entry/shondalan...,Who's mad though?,2017-05-19
54751,POLITICS,Congressional Office Building Has Lead-Tainted...,Arthur Delaney,https://www.huffingtonpost.com/entry/cannon-le...,Capitol Hill workers have been told not to dri...,2016-06-29
130121,PARENTING,What the Olympics Can Teach Our Kids,"Daniel B. Peters, Ph.D., Contributor\nCo-Found...",https://www.huffingtonpost.com/entry/what-the-...,"Of course, scheduling regular family dinners t...",2014-02-21
24796,COMEDY,One Brave Man Tests The Romper Look On The Str...,Steven Hoffer,https://www.huffingtonpost.com/entry/man-rompe...,Is the world ready?,2017-06-05
62623,BUSINESS,Poison Profits,,https://www.huffingtonpost.comhttp://huff.to/1...,Lead paint is making New York City’s children ...,2016-03-31
78298,POLITICS,Elizabeth Warren's Next Target Is Big Oil,Zach Carter,https://www.huffingtonpost.com/entry/elizabeth...,The senator is speaking out against lifting th...,2015-10-06


In [49]:
#Get iterable url list from the dataframe
sample_url_list = sample_dataframe['link'].tolist()

In [50]:
def get_browser():
    # Set up Splinter
    executable_path = {'executable_path': ChromeDriverManager().install()}
    return Browser('chrome', **executable_path, headless=False)

In [51]:
# Set up browser instance
browser = get_browser()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/togneva/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


In [52]:
# Create an empty list to hold the url data.
article_list = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the urls in the list.
for i, url in enumerate(sample_url_list):

    # Group urls in sets of 10 for logging purposes.
    if (i % 10 == 0 and i >= 10):
        set_count += 1
        record_count = 1
        time.sleep(5)

    # Log the URL, record, and set numbers.
    print(f"Processing Record {record_count} of Set {set_count}")
    # Add 1 to the record count.
    record_count += 1
    
# Scrape each url in the list and append to the list of articles
    try:
        #Visit the news article site
        browser.visit(url)

        # Optional delay for loading the page
        browser.is_element_present_by_css('div.list_text', wait_time=1)

        # Convert the browser html to a soup object and then quit the browser
        html = browser.html
        news_soup = soup(html, 'html.parser')

        cli_article = ""
        cli_article_bites = news_soup.find_all('div', class_='primary-cli cli cli-text')
        for cli in cli_article_bites:
            cli_article+= cli.text + '\n'
            
            # Append the article to the list of articles
        article_list.append(cli_article)
    #         print(cli_article)
        
        
# If an error is experienced, skip the url and add a record of scraping error to the article list.
    except Exception:
        article_list.append('article could not be scraped')
        print(f"Url {url} can not be scraped. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1
Processing Record 2 of Set 1
Processing Record 3 of Set 1
Processing Record 4 of Set 1
Processing Record 5 of Set 1
Url https://www.huffingtonpost.com/entry/shondalands-new-show-for-the-people-looks-a-lot-like-htgawm_us_591f055ae4b034684b0bd7b5 can not be scraped. Skipping...
Processing Record 6 of Set 1
Processing Record 7 of Set 1
Processing Record 8 of Set 1
Processing Record 9 of Set 1
Url https://www.huffingtonpost.comhttp://huff.to/1T7XWTe can not be scraped. Skipping...
Processing Record 10 of Set 1
Processing Record 1 of Set 2
Processing Record 2 of Set 2
Processing Record 3 of Set 2
Processing Record 4 of Set 2
Processing Record 5 of Set 2
Processing Record 6 of Set 2
Processing Record 7 of Set 2
Processing Record 8 of Set 2
Processing Record 9 of Set 2
Processing Record 10 of Set 2
Url https://www.huffingtonpost.com/entry/california-beef-recall-cancer_us_5b9de659e4b03a1dcc8e5dac can not b

In [53]:
len(article_list)

30

In [54]:
sample_dataframe['article'] = article_list

In [55]:
article_list

["Has the world distracted you from your unique, individual, potential and core self? Dr. Phil has a one-word answer that almost certainly lies at the heart of why you got off track in the first place and have continued there: fear. Fear of change is common currency among human beings. And not only are you fearful of changing, others have all sorts of fears about you changing. The ruts we get into, no matter how damaging or limiting, are at least familiar and predictable. Stepping up and out of those ruts is scary.\nReady to face the bogeyman and get him out into the light of day? Assume that you will have fear — and then take it apart. What fears have kept you from changing in the past? What kind of fear do you expect to feel when you change? Answer those two questions as they relate to these key words below. Be specific and as honest as you know how to be.\nFamily God Children Spouse Friends Job \nThen, ask yourself: What common themes do you see? Fear of losing? Not being loved or r

In [56]:
sample_dataframe['article_length'] = sample_dataframe['article'].str.len()

In [57]:
sample_dataframe.head()

,category,headline,authors,link,short_description,date,article,article_length
95507,ENTERTAINMENT,One Thing That's Holding You Back From Getting...,Rochelle Nelson,https://www.huffingtonpost.com/entry/how-to-fa...,,2015-03-23,"Has the world distracted you from your unique,...",1497
272,TECH,Instagram Is Adding A 'Mute' Button For The Sa...,Sara Boboltz,https://www.huffingtonpost.com/entry/instagram...,And no one will know when you use it.,2018-05-22,Instagram users will begin to notice a new fea...,1209
38678,POLITICS,The Biggest Political Winners And Losers Of 2016,"Travis Irvine, ContributorComedian, journalist...",https://www.huffingtonpost.com/entry/the-bigge...,What victories should be hailed and what mista...,2016-12-29,As the wildest political year in recent memory...,5869
111783,POLITICS,A GOP Candidate Shared Thoughts About Medical ...,Kim Bellware,https://www.huffingtonpost.com/entry/bruce-rau...,,2014-09-17,"Illinois' medical marijuana law ""stinks"" and s...",4183
26180,BLACK VOICES,Shondaland's New Show 'For The People' Looks A...,Zahara Hill,https://www.huffingtonpost.com/entry/shondalan...,Who's mad though?,2017-05-19,article could not be scraped,28
